# 50 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-11-30 07:44:44.435345: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 07:44:44.469701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 07:44:44.469748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 07:44:44.470972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 07:44:44.476952: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 07:44:44.477901: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [1]:
pip install mne

  Obtaining dependency information for mne from https://files.pythonhosted.org/packages/6d/72/7d8ce176e042580a6261c7af0af6d4bbd3b495d304d8c73a229365f52ede/mne-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 655.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
no_of_patients=50

In [3]:
train=sorted(glob('files/*.edf'))
train=train[:no_of_patients]
train

['files/S001R03.edf',
 'files/S002R03.edf',
 'files/S003R03.edf',
 'files/S004R03.edf',
 'files/S005R03.edf',
 'files/S006R03.edf',
 'files/S007R03.edf',
 'files/S008R03.edf',
 'files/S009R03.edf',
 'files/S010R03.edf',
 'files/S011R03.edf',
 'files/S012R03.edf',
 'files/S013R03.edf',
 'files/S014R03.edf',
 'files/S015R03.edf',
 'files/S016R03.edf',
 'files/S017R03.edf',
 'files/S018R03.edf',
 'files/S019R03.edf',
 'files/S020R03.edf',
 'files/S021R03.edf',
 'files/S022R03.edf',
 'files/S023R03.edf',
 'files/S024R03.edf',
 'files/S025R03.edf',
 'files/S026R03.edf',
 'files/S027R03.edf',
 'files/S028R03.edf',
 'files/S029R03.edf',
 'files/S030R03.edf',
 'files/S031R03.edf',
 'files/S032R03.edf',
 'files/S033R03.edf',
 'files/S034R03.edf',
 'files/S035R03.edf',
 'files/S036R03.edf',
 'files/S037R03.edf',
 'files/S038R03.edf',
 'files/S039R03.edf',
 'files/S040R03.edf',
 'files/S041R03.edf',
 'files/S042R03.edf',
 'files/S043R03.edf',
 'files/S044R03.edf',
 'files/S045R03.edf',
 'files/S0

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [5]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [6]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [7]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [8]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(600000, 74950, 600000, 74950)

In [9]:
print(xtest.iloc[:,-1].values)

[-5.7e-05 -4.5e-05 -2.9e-05 ...  9.0e-06  4.0e-06  1.6e-05]


In [10]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [11]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [12]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [13]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_37312/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_37312/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_37312/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [14]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [15]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.65619
[1]	validation_0-mlogloss:3.53640
[2]	validation_0-mlogloss:3.45357
[3]	validation_0-mlogloss:3.38939
[4]	validation_0-mlogloss:3.33728
[5]	validation_0-mlogloss:3.29218
[6]	validation_0-mlogloss:3.25401
[7]	validation_0-mlogloss:3.22164
[8]	validation_0-mlogloss:3.19184
[9]	validation_0-mlogloss:3.16235
[10]	validation_0-mlogloss:3.13872
[11]	validation_0-mlogloss:3.11597
[12]	validation_0-mlogloss:3.09707
[13]	validation_0-mlogloss:3.07800
[14]	validation_0-mlogloss:3.06319
[15]	validation_0-mlogloss:3.04546
[16]	validation_0-mlogloss:3.02813
[17]	validation_0-mlogloss:3.01325
[18]	validation_0-mlogloss:2.99873
[19]	validation_0-mlogloss:2.98326
[20]	validation_0-mlogloss:2.97057
[21]	validation_0-mlogloss:2.95850
[22]	validation_0-mlogloss:2.94614
[23]	validation_0-mlogloss:2.93830
[24]	validation_0-mlogloss:2.92912
[25]	validation_0-mlogloss:2.91964
[26]	validation_0-mlogloss:2.90740
[27]	validation_0-mlogloss:2.89832
[28]	validation_0-mlogloss:2.8

In [17]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [18]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10


18750/18750 [==============================] - 28s 1ms/step - loss: 2.4155 - val_loss: 2.4537
Epoch 2/10
18750/18750 [==============================] - 25s 1ms/step - loss: 2.0203 - val_loss: 2.3376
Epoch 3/10
18750/18750 [==============================] - 26s 1ms/step - loss: 1.7825 - val_loss: 2.2107
Epoch 4/10
18750/18750 [==============================] - 26s 1ms/step - loss: 1.6656 - val_loss: 2.3141
Epoch 5/10
18750/18750 [==============================] - 27s 1ms/step - loss: 1.6130 - val_loss: 2.2517
Epoch 6/10
18750/18750 [==============================] - 25s 1ms/step - loss: 1.5797 - val_loss: 2.2798
Epoch 7/10
18750/18750 [==============================] - 25s 1ms/step - loss: 1.5552 - val_loss: 2.3051
Epoch 8/10
18750/18750 [==============================] - 25s 1ms/step - loss: 1.5373 - val_loss: 2.3609
Epoch 9/10
18750/18750 [==============================] - 25s 1ms/step - loss: 1.5238 - val_loss: 2.3464
Epoch 10/10
18750/18750 [==============================] - 25s 1ms

In [19]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

  60/2343 [..............................] - ETA: 1s  

2343/2343 [==============================] - 2s 832us/step
              precision    recall  f1-score   support

           0       0.20      0.13      0.16      1499
           1       0.24      0.15      0.18      1499
           2       0.34      0.18      0.23      1499
           3       0.60      0.54      0.57      1499
           4       0.76      0.65      0.70      1499
           5       0.53      0.49      0.51      1499
           6       0.41      0.61      0.49      1499
           7       0.69      0.49      0.57      1499
           8       0.51      0.54      0.52      1499
           9       0.50      0.33      0.40      1499
          10       0.31      0.24      0.27      1499
          11       0.19      0.42      0.26      1499
          12       0.34      0.15      0.21      1499
          13       0.16      0.16      0.16      1499
          14       0.25      0.10      0.14      1499
          15       0.42      0.35      0.38      1499
          16       0.1

## 0-16

In [16]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [17]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_37312/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_37312/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_37312/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.000025,0.000034,0.000019,0.000012,0.000017,0.000013,0.000018,0.000029,0.000014,0.000010,...,0.000020,0.000008,-0.000002,0.000019,0.000013,0.000005,0.000013,0.000007,0.000006,-0.000060
599996,0.000061,0.000068,0.000057,0.000048,0.000050,0.000044,0.000043,0.000048,0.000049,0.000043,...,0.000013,0.000010,-0.000007,0.000011,0.000012,0.000003,0.000013,0.000020,0.000016,-0.000027
599997,0.000066,0.000065,0.000059,0.000050,0.000051,0.000051,0.000054,0.000031,0.000040,0.000035,...,-0.000002,-0.000025,-0.000032,-0.000006,-0.000002,-0.000013,-0.000019,-0.000010,-0.000002,-0.000059
599998,0.000081,0.000087,0.000066,0.000059,0.000069,0.000060,0.000064,0.000054,0.000043,0.000041,...,0.000011,0.000000,-0.000006,0.000014,0.000014,-0.000003,0.000005,0.000000,0.000005,-0.000064


In [18]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [19]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.58298
[1]	validation_0-mlogloss:3.43123
[2]	validation_0-mlogloss:3.32472
[3]	validation_0-mlogloss:3.24059
[4]	validation_0-mlogloss:3.17167
[5]	validation_0-mlogloss:3.11263
[6]	validation_0-mlogloss:3.06031
[7]	validation_0-mlogloss:3.01581
[8]	validation_0-mlogloss:2.97464
[9]	validation_0-mlogloss:2.93529
[10]	validation_0-mlogloss:2.90282
[11]	validation_0-mlogloss:2.87252
[12]	validation_0-mlogloss:2.84612
[13]	validation_0-mlogloss:2.81776
[14]	validation_0-mlogloss:2.79218
[15]	validation_0-mlogloss:2.77145
[16]	validation_0-mlogloss:2.74817
[17]	validation_0-mlogloss:2.72689
[18]	validation_0-mlogloss:2.70555
[19]	validation_0-mlogloss:2.68634
[20]	validation_0-mlogloss:2.66898
[21]	validation_0-mlogloss:2.65002
[22]	validation_0-mlogloss:2.63356
[23]	validation_0-mlogloss:2.61772
[24]	validation_0-mlogloss:2.60055
[25]	validation_0-mlogloss:2.58534
[26]	validation_0-mlogloss:2.57193
[27]	validation_0-mlogloss:2.56120
[28]	validation_0-mlogloss:2.5

In [23]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [24]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10


2023-10-01 12:42:23.360328: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 38400000 exceeds 10% of free system memory.


18750/18750 [==============================] - 26s 1ms/step - loss: 1.6465 - val_loss: 2.0962
Epoch 2/10
18750/18750 [==============================] - 26s 1ms/step - loss: 1.0031 - val_loss: 2.2111
Epoch 3/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.8547 - val_loss: 2.0914
Epoch 4/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.7681 - val_loss: 2.0955
Epoch 5/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.7143 - val_loss: 2.0314
Epoch 6/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.6766 - val_loss: 2.1990
Epoch 7/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.6468 - val_loss: 2.1870
Epoch 8/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.6237 - val_loss: 2.2895
Epoch 9/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.6036 - val_loss: 2.2044
Epoch 10/10
18750/18750 [==============================] - 25s 1ms

In [25]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

2343/2343 [==============================] - 2s 822us/step
              precision    recall  f1-score   support

           0       0.48      0.42      0.44      1499
           1       0.45      0.37      0.41      1499
           2       0.24      0.31      0.27      1499
           3       0.47      0.61      0.53      1499
           4       0.85      0.85      0.85      1499
           5       0.66      0.69      0.68      1499
           6       0.76      0.91      0.83      1499
           7       0.84      0.72      0.77      1499
           8       0.44      0.78      0.56      1499
           9       0.60      0.68      0.64      1499
          10       0.42      0.14      0.21      1499
          11       0.67      0.54      0.60      1499
          12       0.50      0.38      0.43      1499
          13       0.25      0.53      0.34      1499
          14       0.28      0.47      0.35      1499
          15       0.65      0.24      0.35      1499
          16       0.1

## 0-32

In [20]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [21]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_37312/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_37312/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_37312/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [22]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [23]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.36006
[1]	validation_0-mlogloss:3.12523
[2]	validation_0-mlogloss:2.97498
[3]	validation_0-mlogloss:2.85641
[4]	validation_0-mlogloss:2.76006
[5]	validation_0-mlogloss:2.68293
[6]	validation_0-mlogloss:2.61182
[7]	validation_0-mlogloss:2.54959
[8]	validation_0-mlogloss:2.49145
[9]	validation_0-mlogloss:2.44333
[10]	validation_0-mlogloss:2.39608
[11]	validation_0-mlogloss:2.35553
[12]	validation_0-mlogloss:2.31693
[13]	validation_0-mlogloss:2.28224
[14]	validation_0-mlogloss:2.24607
[15]	validation_0-mlogloss:2.21522
[16]	validation_0-mlogloss:2.18454
[17]	validation_0-mlogloss:2.15732
[18]	validation_0-mlogloss:2.13137
[19]	validation_0-mlogloss:2.10774
[20]	validation_0-mlogloss:2.08460
[21]	validation_0-mlogloss:2.06076
[22]	validation_0-mlogloss:2.03857
[23]	validation_0-mlogloss:2.01797
[24]	validation_0-mlogloss:1.99674
[25]	validation_0-mlogloss:1.97802
[26]	validation_0-mlogloss:1.96037
[27]	validation_0-mlogloss:1.94096
[28]	validation_0-mlogloss:1.9

In [29]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [30]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10


2023-10-01 12:46:40.351159: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76800000 exceeds 10% of free system memory.


18750/18750 [==============================] - 26s 1ms/step - loss: 1.1356 - val_loss: 1.8813
Epoch 2/10
15529/18750 [=======================>......] - ETA: 4s - loss: 0.4705

18750/18750 [==============================] - 26s 1ms/step - loss: 0.4570 - val_loss: 1.8390
Epoch 3/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.3354 - val_loss: 1.6959
Epoch 4/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.2775 - val_loss: 1.8002
Epoch 5/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.2420 - val_loss: 1.6499
Epoch 6/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.2172 - val_loss: 1.6240
Epoch 7/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1991 - val_loss: 1.6648
Epoch 8/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.1846 - val_loss: 1.7991
Epoch 9/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1747 - val_loss: 1.8616
Epoch 10/10
18750/18750 [==============================] - 25s 1ms/step - loss: 0.1666 - val_loss: 1.7579


In [31]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

2343/2343 [==============================] - 2s 842us/step
              precision    recall  f1-score   support

           0       0.87      0.54      0.67      1499
           1       0.62      0.80      0.70      1499
           2       0.62      0.60      0.61      1499
           3       0.54      0.62      0.57      1499
           4       0.89      0.94      0.91      1499
           5       0.84      0.94      0.89      1499
           6       0.97      0.88      0.92      1499
           7       0.87      0.77      0.82      1499
           8       0.36      0.87      0.51      1499
           9       0.71      0.80      0.75      1499
          10       0.79      0.90      0.84      1499
          11       0.90      0.97      0.93      1499
          12       0.65      0.76      0.70      1499
          13       0.64      0.85      0.73      1499
          14       0.48      0.75      0.59      1499
          15       0.97      0.09      0.17      1499
          16       0.1

## 0-64

In [12]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [13]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [14]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.05913
[1]	validation_0-mlogloss:2.76593
[2]	validation_0-mlogloss:2.59051
[3]	validation_0-mlogloss:2.45501
[4]	validation_0-mlogloss:2.33595
[5]	validation_0-mlogloss:2.24296
[6]	validation_0-mlogloss:2.15791
[7]	validation_0-mlogloss:2.09009
[8]	validation_0-mlogloss:2.02049
[9]	validation_0-mlogloss:1.96296
[10]	validation_0-mlogloss:1.91300
[11]	validation_0-mlogloss:1.86588
[12]	validation_0-mlogloss:1.82345
[13]	validation_0-mlogloss:1.78351
[14]	validation_0-mlogloss:1.74389
[15]	validation_0-mlogloss:1.70949
[16]	validation_0-mlogloss:1.67419
[17]	validation_0-mlogloss:1.64566
[18]	validation_0-mlogloss:1.61226
[19]	validation_0-mlogloss:1.58593
[20]	validation_0-mlogloss:1.55834
[21]	validation_0-mlogloss:1.53123
[22]	validation_0-mlogloss:1.50894
[23]	validation_0-mlogloss:1.48734
[24]	validation_0-mlogloss:1.46692
[25]	validation_0-mlogloss:1.44579
[26]	validation_0-mlogloss:1.42670
[27]	validation_0-mlogloss:1.40704
[28]	validation_0-mlogloss:1.3

In [34]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [35]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10


2023-10-01 12:51:00.484688: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 153600000 exceeds 10% of free system memory.


18742/18750 [============================>.] - ETA: 0s - loss: 0.7319

18750/18750 [==============================] - 27s 1ms/step - loss: 0.7317 - val_loss: 1.6029
Epoch 2/10
18750/18750 [==============================] - 27s 1ms/step - loss: 0.2781 - val_loss: 1.7273
Epoch 3/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.2062 - val_loss: 1.6858
Epoch 4/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1697 - val_loss: 1.6180
Epoch 5/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1450 - val_loss: 1.7696
Epoch 6/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1274 - val_loss: 1.7156
Epoch 7/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1157 - val_loss: 1.4283
Epoch 8/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.1043 - val_loss: 1.5649
Epoch 9/10
18750/18750 [==============================] - 26s 1ms/step - loss: 0.0968 - val_loss: 1.6063
Epoch 10/10
18750/18750 [==============================] - 26s 1ms

In [36]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

2343/2343 [==============================] - 2s 843us/step
              precision    recall  f1-score   support

           0       0.76      0.77      0.76      1499
           1       0.93      0.85      0.89      1499
           2       0.69      0.67      0.68      1499
           3       0.85      0.95      0.90      1499
           4       0.92      0.98      0.95      1499
           5       0.92      0.98      0.95      1499
           6       0.92      0.97      0.94      1499
           7       0.97      0.48      0.65      1499
           8       0.45      0.82      0.58      1499
           9       0.87      0.69      0.77      1499
          10       0.82      0.69      0.75      1499
          11       0.92      0.92      0.92      1499
          12       0.84      0.74      0.79      1499
          13       0.75      0.94      0.83      1499
          14       0.59      0.87      0.70      1499
          15       0.89      0.51      0.64      1499
          16       0.7